In [2]:
%iam_role arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 0.35 and you have 0.31 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Current iam_role is None
iam_role has been set to arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role.


In [4]:
%region us-west-2

Previous region: None
Setting new region to: us-west-2
Region is set to: us-west-2


In [0]:
%additional_python_modules "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl"
%number_of_workers 2
%connections dev-glue-redshift-connection
%idle_timeout 60

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Additional python modules to be included:
s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl
Previous number of workers: 5
Setting new number of workers to: 2
Connections to be included:
dev-glue-redshift-connection
Current idle_timeout is None minutes.
idle_timeout has been set to 60 minutes.
Authenticating with profile=default
glue_role_arn defined by user: arn:aws:iam::481237842548:role/CF-dev-TruveDevops-Glue-Role
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 2
Session ID: 7343626d-9330-479e-90b2-7fa628d91202
Applying the following default arguments:
--glue_kernel_version 0.31
--enable-glue-datacatalog true
--additional-python-modules s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/sstm_transformation-0.1.0-py3-none-any.whl
Waiting for session 7343626d-9330-479e-90b2-7fa628d91202 to get into ready status...
Session 7343626d-9330-479e-90b2-7fa628d91202 has been created




In [1]:
#download configuration file
import boto3
session = boto3.Session()

s3 = session.resource('s3')
s3.Bucket('dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o').download_file('confs/filevine/sstm.yaml', 'sstm.yaml')

In [2]:
with open("sstm.yaml", "r") as f:
    sstm_config = f.read()
    

In [3]:
def readFromFile(ext, path) :
    fileContent = spark.read.format(ext).option("header", "true").option("mergeSchema", "true").load(path)
    return fileContent


In [4]:
def writeToFile(df_join , path, mode) :
    df_join.write.parquet(path,mode=mode)

In [5]:
spark.conf.set("spark.sql.parquet.enableVectorizedReader","false")
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")


In [6]:
from sstm_transformation.tsm_builder import TSMBuilder
from pyspark.sql.functions import input_file_name
import pyspark.sql.functions as F



In [7]:
#Read All dfs 

#df_project =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/project.parquet")
#df_intake =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/intake.parquet")
#df_contact =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/contact/historical_contacts.parquet")
#df_projecttype = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/projecttype/")
df_meds = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/collections/meds.parquet")
#df_casesummary =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/form/casesummary.parquet")
#df_project_vitals =readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/*/project_vitals.parquet")


In [9]:
df_meds.count()

28153


In [8]:
df_project_vitals = df_project_vitals.withColumn("input_file", input_file_name())


In [10]:
df_meds_modified = df_meds.filter(df_meds.id.isNotNull())

In [11]:
df_meds_modified.count()

16117


In [9]:
df_project_vitals = df_project_vitals.withColumn('projectId', F.element_at(F.split(F.col('input_file'), '/'), -2))


In [12]:
builder = TSMBuilder("sstm.yaml", spark=spark)


In [11]:
df_project_vitals.show()
#s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/18764/10387917/project_vitals.parquet

+--------------------+--------------------+-----------+--------------------+--------+-----+--------------------+---------+
|           fieldName|        friendlyName|  fieldType|               value|position|links|          input_file|projectId|
+--------------------+--------------------+-----------+--------------------+--------+-----+--------------------+---------+
|paralegal223935Fu...|        Case Manager|    Default|       Mark Patricio|       0|   {}|s3://dev-truve-de...|  7866965|
|        incidentDate|       Incident Date|   DateOnly|2020-10-05T00:00:...|       1|   {}|s3://dev-truve-de...|  7866965|
|statuteoflimitati...|                 SOL|   DateOnly|2022-10-05T00:00:...|       2|   {}|s3://dev-truve-de...|  7866965|
|     clientBirthDate|Client Date of Birth|   DateOnly|1978-02-09T00:00:...|       3|   {}|s3://dev-truve-de...|  7866965|
|       mridone223935|           MRI Done?|    Boolean|                null|       4|   {}|s3://dev-truve-de...|  7866965|
|  accidenttype2

In [15]:
#Upload data on S3 
from awsglue.dynamicframe import DynamicFrame
def upload_result(result):
    temp_bucket = "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/temp"
    for key, df in result.items():
        print(f"Writing {key}")
        writeToFile(df,"s3://dev-truve-devops-05-databr-bucketetlprocesseddata-h2m2xopoctot/{}/".format(key.lower()), "overwrite")
        dyf = DynamicFrame.fromDF(dataframe=df, glue_ctx=glueContext, name=key)
        table_name = key.lower()
        dyf.show(10)
        
        datasink = glueContext.write_dynamic_frame.from_jdbc_conf(
                        frame=dyf,
                        catalog_connection="dev-glue-redshift-connection",
                        connection_options={
                            "preactions" : "delete from tsm_dev.{}".format(table_name),
                            "dbtable": "tsm_dev.{}".format(table_name), 
                            "database": "truve",
                        },
                        redshift_tmp_dir=temp_bucket,
                        transformation_ctx="datasink",
                    )
        '''
        #break
        redshift_connection_options = {"url": "jdbc:redshift://dev-redshift-cluster.cxorr0thbkvd.us-west-2.redshift.amazonaws.com:5439/truve", 
                        "user": "truveadmin", 
                        "password": "XXXXXXX",
                        "preactions" : "delete from tsm.{}".format(table_name),
                        "dbtable": "tsm.{}".format(table_name), 
                        "redshiftTmpDir": temp_bucket} 
        datasink4 = glueContext.write_dynamic_frame.from_options(frame=dyf,
                                                        connection_type="redshift",
                                                        connection_options=redshift_connection_options)
        '''
        #break
    

In [ ]:
#PeopleType Transformation
def people_type_transformation(df_contact):
    #Read Raw Data
    result = builder.build_peopletypes(contact_df=df_contact)
    #print(result)
    upload_result(result)
    
people_type_transformation(df_contact)

In [ ]:
#Peoples Transformation
def people_master_transformation(df_contact):
    #Read Raw Data
    result = builder.build_peoplemaster(contact_df=df_contact)
    #print(result)
    #return result
    upload_result(result)
    
people_master_result = people_master_transformation(df_contact)

In [ ]:
#Peoples Transformation
def projecttype_transformation(df_projecttype):
    #Read Raw Data
    result = builder.build_practicetypes(df_projecttype)
    print(result)
    upload_result(result)
    
projecttype_transformation(df_projecttype)

In [ ]:
#Peoples Transformation
df_projecttype = readFromFile("parquet", "s3://dev-truve-devops-05-databricks-bucketetlrawdata-wu3m2thgf3o/filevine/6586/projecttype/")

def phases_transformation(df_projecttype):
    #Read Raw Data
    
    result = builder.build_phasemaster(df_projecttype)
    print(result)
    upload_result(result)
    
phases_transformation(df_projecttype)

In [16]:
#Peoples Transformation
def value_transformation(df_meds):
    #Read Raw Data
    result = builder.build_casefigures(df_meds)
    #print(result)
    upload_result(result)
    return result
    
result = value_transformation(df_meds_modified)

+------------+-------------+--------------+--------+--------------------+--------------------+--------------------+-----------+-------------+--------+-------+-------+-------+
|Truve_Org_ID|Client_Org_ID|Parent_Case_ID| Case_ID|      Case_Figure_ID|         Figure_Type|     Figure_Sub_Type|Figure_Date|Figure_Status|   Value|Custom1|Custom2|Custom3|
+------------+-------------+--------------+--------+--------------------+--------------------+--------------------+-----------+-------------+--------+-------+-------+-------+
|        6586|         6586|       8865186| 8865186|585273e8-e51f-48f...|       Case Expenses|       Case Expenses|       null|         null|   108.0|   null|   null|   null|
|        6586|         6586|       8865186| 8865186|63142579-4e07-4ea...|       Case Expenses|       Case Expenses|       null|         null|    98.0|   null|   null|   null|
|        6586|         6586|       8865186| 8865186|e8d7cf5b-5fbd-47c...|       Case Expenses|       Case Expenses|       nul

In [ ]:
#Peoples Transformation
def cases_transformation(df_project):
    #Read Raw Data
    result = builder.build_casemaster(df_project)
    return result
    #print(result)
    #upload_result(result)
    
result = cases_transformation(df_project)

In [ ]:
#Peoples Transformation
def intake_transformation(df_intake):
    #Read Raw Data
    result = builder.build_intakesummary(df_intake)
    return result 

result = intake_transformation(df_intake)
#result["IntakeDetails"] = result["IntakeDetails"].drop("referral_fee")
upload_result(result)


In [13]:
#Peoples Transformation
def casesummary_transformation(df_casesummary, df_project_vitals, df_project):
    #Read Raw Data
    result = builder.build_casesummary(df_casesummary, df_project_vitals, df_project)
    print(result)
    upload_result(result)
    
casesummary_transformation(df_casesummary, df_project_vitals, df_project)

+--------+------------+-------------+--------------+-------------------+---------------------+----------------+-----------------------+-------------------------+---------------------------+-----------------------+-----------+--------------------+------------+-----------+--------------------------+-----------------------+--------------+------------+------------+------------+--------------+-------------+-------+-------+-------+-------+-------+--------------------+
| Case_ID|Truve_Org_ID|Client_Org_ID|Parent_Case_ID|Plaintiff_Full_Name|Case_Marketing_Source|Case_Source_Name|Attorney_Fee_Percentage|Estimated_Settlement_Date|Estimated_Settlement_Amount|If_Case_Settled_Presuit|If_VIP_Case|If_Case_Referred_Out|Case_Type_ID|Attorney_ID|Prelitigation_Paralegal_ID|Litigation_Paralegal_ID|CaseManager_ID|Cocounsel_ID|Case_Team_ID|Insurance_ID|Case_Status_ID|Case_Phase_ID|Custom1|Custom2|Custom3|Custom4|Custom5|    Case_Create_Date|
+--------+------------+-------------+--------------+--------------

In [ ]:
processed_project_df = result["CMS_Cases"]
processed_project_df.printSchema()

In [ ]:
#Peoples Transformation
def casetype_transformation(df_casesummary, project_df):
    #Read Raw Data
    result = builder.build_casetypes(df_casesummary, project_df)
    #print(result)
    #return result
    upload_result(result)
    return result
    
casetype_result = casetype_transformation(df_casesummary, df_project)

In [ ]:
processed_case_type_df = casetype_result["CMS_CaseTypes"]

#Peoples Transformation
def cases_transformation(df_project, processed_case_type_df, df_project_vitals, df_casesummary):
    #Read Raw Data
    result = builder.build_casemaster(df_project, processed_case_type_df, df_project_vitals, df_casesummary)
    #return result
    #print(result)
    upload_result(result)
    return result
    
result = cases_transformation(df_project, processed_case_type_df, df_project_vitals, df_casesummary)

In [ ]:
%stop_session

In [ ]:
df_casesummary.select("caseType").distinct().show(10)

In [ ]:
casetype_df = casetype_result["CMS_CaseTypes"]


In [ ]:
casetype_df.show()

In [ ]:
people_df = people_master_result["CMS_People"]

In [ ]:
people_df

In [ ]:
people_df.show()

In [ ]:

# Find count for empty, None, Null, Nan with string literals.
from pyspark.sql.functions import col,isnan,when,count
df2 = people_df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() , c 
                           )).alias(c)
                    for c in people_df.columns])
df2.show()


In [ ]:
%stop_session

In [ ]:
case_df = result["CMS_Cases"]

In [ ]:
case_figure_df.select("Case_Figure_ID", "Case_ID").show()

In [ ]:
from pyspark.sql.functions import col,isnan,when,count
df2 = case_df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() , c 
                           )).alias(c)
                    for c in case_df.columns])
df2.show()
